In [4]:
import pandas as pd
import re
import numpy as np
import os # <--- NEW IMPORT for path handling

def dms_to_dd(dms_str):
    """
    Converts a DMS string (e.g., '8°15\' S', '29 45 E') to a Decimal Degree (DD) float.
    Handles degrees, minutes, and seconds, and uses the cardinal direction (S/W)
    to determine the sign.
    """
    if pd.isna(dms_str):
        return np.nan

    # Normalize the string: remove symbols, convert to uppercase, and replace 'TO'
    dms_str = str(dms_str).upper().replace('°', ' ').replace('\'', ' ').replace('"', ' ').replace('TO', ' ').strip()
    
    # Extract numbers (degrees, minutes, seconds)
    parts = re.findall(r"(\d+\.?\d*)\s*", dms_str)
    
    # Get the final cardinal direction (N, S, E, W)
    direction_match = re.search(r"([NSEW])$", dms_str)
    direction = direction_match.group(1) if direction_match else ''
    
    # Safely assign D, M, S, using 0 for missing components
    D = float(parts[0]) if len(parts) > 0 else 0.0
    M = float(parts[1]) if len(parts) > 1 else 0.0
    S = float(parts[2]) if len(parts) > 2 else 0.0

    dd = D + M/60.0 + S/3600.0

    # Apply sign for South or West
    if direction in ('S', 'W'):
        dd *= -1
    
    return dd

# ----------------- Main Script (Final File Paths) -----------------

# 1. INPUT FILE PATH (The Excel file you are reading)
INPUT_FILENAME = r"C:\Users\willi\OneDrive\Documents\COVEX\Cleaned 2\For EDA\Name and Coordinates Only.xlsx"

# 2. OUTPUT DIRECTORY (The folder where the new CSV will be saved)
OUTPUT_DIR = r"C:\Users\willi\OneDrive\Documents\COVEX\Cleaned 2\For EDA"

# Construct the full output file path
OUTPUT_FILENAME = os.path.join(OUTPUT_DIR, "Updated_Mining_Properties_Coordinates.csv")


try:
    # Use pd.read_excel for .xlsx files, assuming the data is on 'Sheet1'
    df = pd.read_excel(INPUT_FILENAME, sheet_name='Sheet1')
    
    # Assuming columns 0, 1, 2 are Name, Latitude, Longitude
    if len(df.columns) < 3:
        raise ValueError("Dataframe has fewer than 3 columns. Check sheet name or header settings.")
    
    df.columns = ['NAME', 'Latitude', 'Longitude'] + list(df.columns[3:])

    # --- 1. Coordinate Conversion ---
    df['Latitude_Decimal_Degrees'] = df['Latitude'].apply(dms_to_dd)
    df['Longitude_Decimal_Degrees'] = df['Longitude'].apply(dms_to_dd)

    # --- 2. Name Correction ---
    df['Property_Name_Updated'] = df['NAME'].replace(['MXUSHI COPPER', 'MXUSHI'], ['MKUSHI COPPER', 'MKUSHI'], regex=True)

    # --- 3. Final Formatting and Output ---
    df_output = df[['Property_Name_Updated', 'Latitude_Decimal_Degrees', 'Longitude_Decimal_Degrees', 'NAME', 'Latitude', 'Longitude']].copy()
    df_output.rename(columns={
        'NAME': 'Original_Name',
        'Latitude': 'Original_Latitude_DMS',
        'Longitude': 'Original_Longitude_DMS'
    }, inplace=True)

    # Save the updated spreadsheet to the specified folder
    df_output.to_csv(OUTPUT_FILENAME, index=False)

    print("Success! Data processing complete.")
    print(f"The updated data has been saved to: {OUTPUT_FILENAME}")
    
except FileNotFoundError:
    print(f"Error: The file or directory was not found. Please check both paths:\nInput: {INPUT_FILENAME}\nOutput Dir: {OUTPUT_DIR}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Success! Data processing complete.
The updated data has been saved to: C:\Users\willi\OneDrive\Documents\COVEX\Cleaned 2\For EDA\Updated_Mining_Properties_Coordinates.csv
